In [1]:
import sys
sys.path.append('..')

from aural_travels.data import fma

## 1 Load Data

In [9]:
# Adjust this to your local copy of the FMA dataset.
# Should contain `fma_dataset` and `fma_metadata` directories.
DATA_DIR = '/home/leo/data/fma'

genres = fma.load_genres(DATA_DIR)
tracks = fma.load_tracks(DATA_DIR)

## 2 Genres

In [3]:
# All genres
genres

,#tracks,parent,title,top_level
genre_id,,,,
1,8693,38,Avant-Garde,38
2,5271,0,International,2
3,1752,0,Blues,3
4,4126,0,Jazz,4
5,4106,0,Classical,5
...,...,...,...,...
1032,60,102,Turkish,2
1060,30,46,Tango,2
1156,26,130,Fado,2


In [4]:
# Top-level genres
genres[genres.parent == 0]

,#tracks,parent,title,top_level
genre_id,,,,
2,5271,0,International,2
3,1752,0,Blues,3
4,4126,0,Jazz,4
5,4106,0,Classical,5
8,868,0,Old-Time / Historic,8
9,1987,0,Country,9
10,13845,0,Pop,10
12,32923,0,Rock,12
13,730,0,Easy Listening,13


## 3 Tracks

In [18]:
# The track metadata contains four sets of columns:
# track, album, artist, and set.
tracks.columns.values

array([('album', 'comments'), ('album', 'date_created'),
       ('album', 'date_released'), ('album', 'engineer'),
       ('album', 'favorites'), ('album', 'id'), ('album', 'information'),
       ('album', 'listens'), ('album', 'producer'), ('album', 'tags'),
       ('album', 'title'), ('album', 'tracks'), ('album', 'type'),
       ('artist', 'active_year_begin'), ('artist', 'active_year_end'),
       ('artist', 'associated_labels'), ('artist', 'bio'),
       ('artist', 'comments'), ('artist', 'date_created'),
       ('artist', 'favorites'), ('artist', 'id'), ('artist', 'latitude'),
       ('artist', 'location'), ('artist', 'longitude'),
       ('artist', 'members'), ('artist', 'name'),
       ('artist', 'related_projects'), ('artist', 'tags'),
       ('artist', 'website'), ('artist', 'wikipedia_page'),
       ('set', 'split'), ('set', 'subset'), ('track', 'bit_rate'),
       ('track', 'comments'), ('track', 'composer'),
       ('track', 'date_created'), ('track', 'date_recorded'),
   

In [21]:
tracks['album'].sample(3)

,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type
track_id,,,,,,,,,,,,,
82405,0,2013-04-08 16:06:12,2013-01-16,NaN,1,13989,<p>chris weisman : bass</p>\n<p>sarah smith : ...,2314,NaN,[],Scam b/w Press Dumps,2,Album
60689,0,2012-02-15 21:24:57,2011-04-07,NaN,1,11106,<p>All words by Matt Plunkett and all music by...,2962,NaN,[],MALLL,12,Album
122491,0,2015-07-13 05:03:14,2015-07-14,NaN,0,18543,<p>What happens when two ON.U Sound prodigies ...,8168,NaN,[],OICHO Dubclash,8,Album


In [44]:
tracks[tracks['set', 'subset'] == 'medium']['track', 'genre_top'].nunique()

16